# Travaux pratiques: Reseaux de neurones

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

Dans ce TP nous allons implémenter les differents éléments qui composent un reseau de neurones 

# Data set

Dans la première partie, nous allons essayer de prédire le prix de vente de biens immobiliers de la ville Boston.  
Nous allons pour cela utiliser le classique jeu de données boston house-prices disponible directement dans la librairie de machine learning [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html)  
Commençons donc par importer le dataset:

In [ ]:
from sklearn.datasets import load_boston
dataset = load_boston()

Nous allons donc essayer de prédire le prix des maisons à partir de 13 features présentées [ici](https://scikit-learn.org/stable/datasets/index.html#boston-dataset).   

Commençons par séparer le dataset en 2:
* Un ensemble d'apprentissage pour entrainer le modèle
* Un ensemble de test pour tester le modèle appris

Utilisez la méthode [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) de scikit-learn pour séparer le dataset de la façon suivante:
* X_train -> les features pour entrainer le modèle
* y_train -> les prix à prédire durant l'apprentissage
* X_test -> les features pour tester le modèle
* y_test -> les prix à prédire pour tester le modèle  

In [ ]:
# %load solutions/boston_train_test_split.py
from sklearn.model_selection import train_test_split
import numpy as np

data = np.asarray(dataset.data, dtype='float32')
target = np.asarray(dataset.target.reshape(-1, 1), dtype='float32')

X_train, X_test, y_train, y_test = ...

Parmi les bonnes pratiques permettant d'améliorer l'apprentissage des réseaux de neurones, il est important de normaliser les données en entrée pour obtenir un moyenne proche de 0 et une variance à 1.  
Utilisez la méthode [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) de scikit-learn pour normaliser X_train et X_test ainsi que y_train et y_test:

In [ ]:
# %load solutions/boston_scaler.py
from sklearn import preprocessing
...

# Réseaux de neurones

Nous allons construire nos réseaux de neurones comme des successions de couches.  
Nous distinguerons, similairement aux frameworks les plus courants (Tensorflow, Pytorch, Keras ...) deux types de couches:
* des couches de poids et gradients
* des couches d'activations 

Chaque couche possèdera:
* une méthode forward qui reçoit en entrée un vecteur $x$ et y applique une transformation qui servira d'inputs pour les couches suivantes
* une méthode backward qui reçoit en entrée des gradients et les retropopages vers les couches précédentes. 
* un argument layer_type permettant de définir le type de couche

La classe suivante définit la structure d'une couche:

In [ ]:
class Layer:
    def __init__(self):
        self.layer_type = 'abstract'

    def forward(self, inputs):
        pass
    
    def backward(self, grad):
        pass

Un réseau de neurones sera donc une classe:
* contenant une liste de couches
* possédant une méthode forward (qui sera en fait la composition des méthodes forward de ses couches) qui pour un vecteur x fournira une prédiction
* et une méthode backward (composition des méthodes forward de ses couches) qui rétropopagera les gradients depuis sa sortie jusqu'a ses premières couches.

![NeuralNetwork](layers.png)

# Layers

Nous allons maintenant implémenter les couches de notre réseau.  
Commençons par les couches "linéaires" composées de poids $W$ et de biais $b$.  
Ces couches reçoivent en entrée un vecteur $x$ et retourne en sortie le résultat de $Wx + b$.

Complétez la definition de la classe LinearLayer:
* les poids et les biais serons stockés dans un dictionnaire "params". Les poids seront une matrice $input \times output$ initialisée uniformémént entre $[-0.1, 0.1]$ (pensez à la fonction random.uniform de numpy)
* implémentez la méthode forward (vous pouvez utiliser la fonction np.matmul pour calculer $Wx$), il faudra garder en mémoire le vecteur d'entrées qui sera utilisé lors de l'appel à la méthode backward:

In [ ]:
#%load solutions/layers.py
class Linear(Layer):
    def __init__(self, input_size, output_size):
        super().__init__()
        
        self.layer_type = 'linear'
        self.params ={}
        self.grads = {}
        #weights est la matrice [input x output] contenant les poids de la couche 
        self.params["weights"] = ...
        #biais est un vecteur de dimension: (output_size)
        self.params["bias"] = ...

    def forward(self, inputs):
        # outputs = inputs @ weights + biais
        #Nous devons garder en mémoire les inputs car ils seront utilisés dans la méthode backward
        self.inputs = inputs
        return ...
    
    def backward(self, grad):
        # we need to sum gradients over the batch axis
        self.grads["weights"] = np.matmul(self.inputs.T, grad)
        self.grads["bias"] = np.sum(grad, axis=0)
        return np.matmul(grad, self.params["weights"].T)

Testez votre layer, la céllule suivant ne doit pas renvoyer d'erreur:

In [ ]:
input_size, output_size = 10, 5
X = np.random.rand(input_size)
grads = np.random.rand(10,5)
test_linear_layer = Linear(input_size, output_size)
assert test_linear_layer.forward(X).shape == (5,)
assert test_linear_layer.backward(grads).shape == (10, 10)

## Activation Layers

Nous allons maintenant implémenter les fonctions d'activation utilisées par les neurones.  
Implémentez une fonction d'activation ainsi que sa dérivée (par exemple la fonction ReLU: $f(x) = max(0,x)$:

In [ ]:
# %load solutions/activations.py
def relu(x):
    return ...

def relu_prime(x):
    return ...

Implémentez maintenant les couches d'activations:  
Elles seront initialisées à partir d'une fonction $f$ et de sa dérivée

In [ ]:
#%load  solutions/activation_layer.py
class Activation(Layer):
    def __init__(self, f, f_prime):
        super().__init__()
        self.layer_type = 'activation'
        self.f = f
        self.f_prime = f_prime

    def forward(self, inputs):
        #Nous devons garder en mémoire les inputs car ils seront utilisés dans la méthode backward
        self.intputs = inputs
        return ...
    
    def backward(self, grad):
        """ si y = f(x) et x = g(z)
        alors dy/dz = f'(x) * g'(z)
        Dans notre cas g'(z) correspond aux gradients en entrée
        """
        return self.f_prime(self.intputs) * grad

Testez votre couche d'activation, la cellule suivante ne dois pas renvoyer d'erreur:

In [ ]:
relu_layer = Activation(f=relu, f_prime=relu_prime)

#forward
x = np.array([-1, 5, -2, 6])
y = relu_layer.forward(x)
assert np.array_equal(y, np.array([0, 5, 0, 6]))

#backward
incomming_grads = np.array([-0.1, 0.6, -0.4, -0.1])
grads = relu_layer.backward(incomming_grads)
assert np.array_equal(grads, np.array([-0. ,  0.6, -0. , -0.1]))

## Neural network

Nous allons maintenant définir la classe qui sera utilisée pour représenter nos réseaux de neurones.  
Un réseau de neurones possédera une liste de couches et deux méthodes: forward et backward.  

Complétez la méthode forward et la méthode backward de la classe NeuralNetwork en appellant les methodes forward et backward des couches du réseau:

In [ ]:
# %load solutions/neural_network.py
class NeuralNetwork:
    def __init__(self, layers):
        self.layers = layers

    def forward(self, inputs):
        ...

    def backward(self, grad):
        ...

Nous avons maintenant tous les éléments constituant un réseau de neurones.  
Instanciez un reseau de neurones constitué de 3 couches:
* une couche d'entrée recevant un vecteur de dimension 13 et constituée de 40 neurones
* une couche cachée de 40 neurones
* une couche de sortie constituée d'un seul neurone

In [ ]:
# %load solutions/boston_network.py

Testez votre réseau sur le jeu de test:

In [ ]:
from sklearn.metrics import mean_absolute_error

y_pred = net.forward(X_test)
print(mean_absolute_error(y_pred, y_test))
    
plt.figure(figsize=(20,5))
plt.scatter(range(len(y_test)), scaler.inverse_transform(y_test), label='target')
plt.scatter(range(len(y_test)), scaler.inverse_transform(y_pred), label='prediction')
plt.legend()

# Loss function

Les prédictions du réseau sont mauvaise.  
C'est normal on ne l'a pas encore entrainé.  
Commençons par implémenter la fonction de perte qu'utilisera le réseau de neurones dans son apprentissage.  
Nous sommes ici dans un problème de régression, nous utiliserons l'erreur quadratique moyenne (Mean Squared Error) pour apprendre notre modèle:
$$\sum_{i}^{n}\frac{(f(x_i) - y_i)^2}{n}$$  
Implémentez la fonction de perte ainsi que sa dérivée qui sera utilisée lors de la retro-propagation:

In [ ]:
# %load solutions/mse.py
class MSE():

    def loss(self, y_pred, y_true):
        #returns a scalar
        return ...
    
    def grad(self, y_pred, y_true):
        #returns a tensor of gradients
        return ...

# Optimizer

Nous allons maintenant impémenter la méthode d'optimisation utilisée durant l'apprentissage.
Nous utiliserons ici la descente de gradient stochastique.
La méthode est simple à chaque pas nous allons mettre à jour les paramètres du réseau grace à la formule suivante: $w_{t+1} = w_t - \eta \nabla$ où $\eta$ est le learning rate.

In [ ]:
# %load solutions/sgd.py
class SGD():
    def __init__(self, lr=0.001):
        self.lr = lr
    
    def step(self, net):
        for layer in net.layers:
            if layer.layer_type == 'linear':
                for param, grad in zip(layer.params.values(), layer.grads.values()):
                    ...

# Fit function

Il ne reste plus qu'à implémenter la fonction fit qui réalise une époque d'apprentissage.  
La fonction doit:
* calculer les predictions du réseau sur le batch d'apprentissage
* calculer la loss
* calculer le gradient de la loss
* retropropager les gradients
* realiser un pas d'optimisation
* retourner la perte 

In [ ]:
# %load solutions/fit.py
def fit(net, loss, optimizer, X, y):
    ...
    return prediction_loss

On peut maintant entrainer notre réseau:

In [ ]:
optimizer = SGD(lr=0.01)
loss = MSE()
for i, (x, y) in enumerate(zip(X_train, y_train)):
    training_loss = fit(net, loss, optimizer, np.array([x]), np.array([y]))

Affichez les résultats de votre entrainement:

In [ ]:
y_pred = net.forward(X_test)
print(mean_absolute_error(y_pred, y_test))
    
plt.figure(figsize=(20,5))
plt.scatter(range(len(y_test)), scaler.inverse_transform(y_test), label='target')
plt.scatter(range(len(y_test)), scaler.inverse_transform(y_pred), label='prediction')
plt.legend()

# Digits

Essayons maintenant notre classe sur un problème de classification.  
Nous allons chercher à classifier des chiffres manuscrits à partir des pixels. Nous utiliserons pour cela le dataset [digits de scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_digits

dataset = load_digits()

Affichez les 10 premiers éléments du dataset:

In [ ]:
#load  solutions/digits.py

Similairement au dataset précédent séparer votre jeu de donné en deux et normalisez les vecteurs d'entrées. (Pas la peine cette fois çi de normaliser les y)

In [ ]:
# %load solutions/digits_preprocessing.py
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

data = np.asarray(dataset.data, dtype='float32')
target = np.asarray(dataset.target, dtype='int32')

X_train, X_test, y_train, y_test = ...

...
X_train = ...
X_test = ...

# Cross entropy

Nous sommes cette fois-ci en présence d'un problème de classification.  
La fonction de perte MSE n'est plus apprpriée pour notre problème.  
Nous allons cette fois çi-utiliser la [cross-entropy.](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_loss_function_and_logistic_regression) comme fonction de perte:
$$ L = \frac{1}{N}\sum_{i=1}^{N}y_i \cdot log(\hat{y_i})$$  
Avec $\hat{y_i}$ la prédiction du réseau sur laquelle est appliqué la fonction softmax:
$$softmax_j(x) = \frac{e^x_j}{\sum_{k}e^x_k}$$

In [ ]:
def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

class CrossEntropy():

    def loss(self, logits, y_true):
        logits_for_answers = logits[np.arange(len(logits)), y_true]
        xentropy = - logits_for_answers + np.log(np.sum(np.exp(logits),axis=-1))
        return xentropy



    def grad(self, logits, y_true):
        ones_for_answers = np.zeros_like(logits)
        ones_for_answers[np.arange(len(logits)), y_true] = 1
        y_softmax = softmax(logits)
        return (- ones_for_answers + y_softmax) / logits.shape[0]

Instanciez un reseau de neurones capable de prédire la classe d'une image:

In [ ]:
#%load solutions/digits_net.py
...
net = ...
optimizer = SGD(lr=0.05)

Calculer la précision de votre réseau sur le jeu de test. (Vous pouvez pour cela utiliser la fonction [acuracy_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html) de scikit-learn)

In [ ]:
#%load  solutions/accuracy1.py

La fonction suivante permet de visualiser nos prédiction

In [ ]:
def plot_prediction(net, sample_idx=range(3), classes=range(10)):
    
    for idx in sample_idx:
        plt.figure()
        logits = net.forward(X_test[idx])
        probas = softmax(logits)
        prediction = np.argmax(probas)

        fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(10, 4))

        ax0.imshow(scaler.inverse_transform(X_test[idx]).reshape(8, 8), cmap=plt.cm.gray_r,
                   interpolation='nearest')
        ax0.set_title("True image label: %d" % y_test[idx]);
        ax1.bar(classes, np.eye(len(classes))[y_test[idx]], label='true')
        ax1.bar(classes, probas, label='prediction', color="red")
        ax1.set_xticks(classes)

        ax1.set_title('Output probabilities (prediction: %d)'
                      % prediction)
        ax1.set_xlabel('Digit class')
        ax1.legend()

In [ ]:
plot_prediction(net, sample_idx=range(3))

Nous allons cette fois-çi, utiliser des mini-batchs durant notre apprentissage.  
La fonction suivante permet d' itérer sur des mini-batchs:

In [ ]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=True):
    assert inputs.shape[0] == targets.shape[0]
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]

Et la fonction suivant de réaliser une époque d'apprentissage à l'aide de mini-batchs:

In [ ]:
def fit_one_epoch(X, y_true):
    for x, y in iterate_minibatches(X, y_true, 32):
        training_loss = fit(net, loss, optimizer, x, y)
        
    return training_loss.mean()

Entrainez votre reseau sur 15 époques.  
A chaque époque calculez votre précision sur le jeu de train et sur le jeu de test et affichez les sur un même graphique une fois l'entrainement terminé.

In [ ]:
#%load solutions/plot_accuracy.py

Affichez votre précision sur le jeu de test et visualisez vos prédiction à l'aide de la fonction plot_prediction)

In [ ]:
# %load solutions/accuracy2.py

Essayez d'afficher votre matrice de confusion en vous inspirant de [ce code:](https://scikit-learn.org/dev/auto_examples/classification/plot_digits_classification.html#sphx-glr-auto-examples-classification-plot-digits-classification-py) 

In [ ]:
# %load solutions/confusion_matrix.py

Affichez les exemples sur lesquels le réseau se trompe:

In [ ]:
# %load solutions/plot_errors.py